Import libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers,callbacks
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make directory for each class

In [ ]:
from distutils.dir_util import copy_tree

all_plants=['Potato','Tomato','Pepper']
all_classes=os.listdir('../input/plant-village/PlantVillage/')

for plant in all_plants:
    path = os.path.join("./", plant)
    os.mkdir(path)
    plant_path = os.path.join(path, 'classes')
    os.mkdir(plant_path)
    output_path = os.path.join(path, 'dataset')
    os.mkdir(output_path)
    for class_name in all_classes:
        if plant in class_name:
            print(class_name)
            to_class_path = os.path.join(plant_path, class_name)
            from_class_path = os.path.join('/kaggle/input/plant-village/PlantVillage/', class_name)
            os.mkdir(to_class_path)
            copy_tree(from_class_path, to_class_path)

In the next cell, you can change the (plant) variable to the vegetable that you want to train the model on.

In [ ]:
plant='Pepper'
path = os.path.join('/kaggle/working/', plant)
ds_path = os.path.join(path, 'classes')
output= os.path.join(path,'dataset')

I used split_folders library to split datasets to THREE main folders, each has one type of vegetable (Potato, Tomato, and Pepper).

In [ ]:
# To split dataset into training, validation, and test set
splitfolders.ratio(ds_path,
                   output=output,
                   seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

Define veriables

In [ ]:
img_size=256
batch_size=32

In [ ]:
dirs=[x[1] for x in os.walk(output)]
dirs[0]

Using ImageDataGenerator api, which allows to load the images from the **Output** and augment them.

In [ ]:
for i in range(3):
    img=ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    zoom_range=0.3,
    brightness_range=(0.2,1.0),
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
    )
    generator = os.path.join(output,dirs[0][i])
    globals()[f"{dirs[0][i]}_generator"]= img.flow_from_directory(
        generator,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        class_mode="sparse"
#         ,save_to_dir="C:\\Code\\potato-disease-classification\\training\\AugmentedImages"
    )

Display the classes (folders) in dataset

In [ ]:
train_generator.class_indices

In [ ]:
test_generator.class_indices

In [ ]:
val_generator.class_indices

In [ ]:
class_names = list(train_generator.class_indices.keys())
class_names

Build the model

In [ ]:
model=models.Sequential([
    layers.Conv2D(32,(3,3), activation='relu',input_shape=(img_size,img_size,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(len(class_names),activation='softmax')
])

model.build(input_shape=(batch_size,img_size,img_size,3))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

Early Stopping

In [ ]:
es=callbacks.EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=1)

Fit the model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    batch_size=32,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1,
    epochs=10
    ,callbacks=[es]
)

Evaluate the model

In [ ]:
score = model.evaluate(test_generator)

Plotting the history of the training model

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
plt.plot( loss,label='Train loss')
plt.plot( val_loss,label='Validation loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
plt.plot(acc,label='Train Accuracy')
plt.plot( val_acc,label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

Prediction using test dataset

In [ ]:
# prediction function
def pred_img(img):
#     add one mor dim for image array
    img_array=tf.expand_dims(img,0)

#     predict the image
    pred=model.predict(img_array)

#     Select the class of the predicted image
    pred_class=class_names[np.argmax(pred[0])]
#     Accuracy of the prediction
    confidence=round((np.max(pred[0])*100),2)

    return pred_class, confidence

In [ ]:
plt.figure(figsize=(40,40))

batch__size=len(test_generator.__getitem__(1)[0])
for i in range (0,batch__size):
#     image array in test_generator first batch
    image=test_generator.__getitem__(1)[0][i]
#     label number in test_generator first batch
    label=test_generator.__getitem__(1)[1][i]
#    predict the image and get the predicted class and the accuracy
    pred_class, confidence = pred_img(image)
#    the actual class of the image
    actual_class= class_names[int(label)]
#    plot the image with its info
    col=int(batch__size/4)
    ax=plt.subplot(col,4,i+1)
    plt.imshow(image)
    plt.title(
        f'Actual: {actual_class}.\nPredicted: {pred_class}.\nAccuracy: {confidence}'
    )
    plt.axis('off')

Save the model

In [ ]:
# model.save(f'models/{plant}')

Delete the plants folders

In [ ]:
import shutil
all_plants=['Potato','Tomato','Pepper']
for plant in all_plants:
    shutil.rmtree(f"/kaggle/working/{plant}")